In [1]:
import ast

# Your Python code as a string
python_code = """
def add_numbers(a, b):
    return a + b
"""

# Parse the code and generate the AST
parsed_ast = ast.parse(python_code)

# Now you can work with the AST
print(ast.dump(parsed_ast))


Module(body=[FunctionDef(name='add_numbers', args=arguments(posonlyargs=[], args=[arg(arg='a'), arg(arg='b')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Return(value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())))], decorator_list=[])], type_ignores=[])


In [12]:
import ast
import sys

class ASTGrapher(ast.NodeVisitor):
    def __init__(self):
        self.graph = []
        self.node_count = 0

    def visit(self, node, parent_name=""):
        current_node_name = f"node{self.node_count}"
        self.node_count += 1

        if isinstance(node, ast.AST):
            self.graph.append(f'{current_node_name} [label="{type(node).__name__}"]')
        elif isinstance(node, ast.expr):
            lexeme = f"{ast.get_source_segment(node)}"
            self.graph.append(f'{current_node_name} [label="{lexeme}"]')
        else:
            self.graph.append(f'{current_node_name} [label="{type(node).__name__}"]')

        if parent_name:
            self.graph.append(f"{parent_name} -> {current_node_name}")

        for child_name, child_node in ast.iter_fields(node):
            if isinstance(child_node, list):
                for child in child_node:
                    if isinstance(child, ast.AST):
                        self.visit(child, current_node_name)
                    elif isinstance(child, ast.expr):
                        self.visit(child, current_node_name)
            elif isinstance(child_node, ast.AST):
                self.visit(child_node, current_node_name)
            elif isinstance(child_node, ast.expr):
                self.visit(child_node, current_node_name)

    def generate_dot(self, code):
        parsed_ast = ast.parse(code)
        self.visit(parsed_ast)
        return "digraph AST {\n" + "\n".join(self.graph) + "\n}"

# # Your Python code as a string
# python_code = """
# def add_numbers(a, b):
#     return a + b
# """

grapher = ASTGrapher()
dot_code = grapher.generate_dot(python_code)

# Save the DOT code to a file
with open("ast_graph.dot", "w") as dot_file:
    dot_file.write(dot_code)

print("DOT code generated successfully. You can use Graphviz tools to visualize the AST.")


DOT code generated successfully. You can use Graphviz tools to visualize the AST.
